In [12]:
import numpy as np
from tensorflow.keras.datasets import cifar10
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data() #هنا انا عملت  داونلود للدتا عادي

features = np.concatenate((train_images, test_images), axis=0)#هنا انا مخلي الصور لوحدها
labels = np.concatenate((train_labels, test_labels), axis=0)#وهنا بردوا الكلاسيس لوحدها عادي

In [13]:
features.shape#طبعا احنا مش هينفع تشتغل ع الشكل ده عشان ده معناه ان عندك 60000صورة كل واحدة سايز 32*32*3
# ف هنضطر نغيرها يعنيRGB

(60000, 32, 32, 3)

In [14]:
labels.shape# هنا دول عدد الكلاسيس 600000مظبوط ع الصور عادي

(60000, 1)

In [15]:
features=features.reshape((features.shape[0],-1))# هنا انا حولتلك الصور ام
#rgb لفيشترز عشان تقدر تشوفها
#2d عشان تقدر تتخيلهم كل عمود هو عبارة عن بيكسل من 0 لحد255

In [16]:
labels# هنا طبعا دول 60000واحدة متكررين عشان انا جمعتهم كلهم ع بعض فوق لكن الكلاسيس عندك من 0 لحد 9

array([[6],
       [9],
       [9],
       ...,
       [5],
       [1],
       [7]], dtype=uint8)

In [17]:
features.shape

(60000, 3072)

In [18]:
features = features/255.0

In [19]:
type(features)

numpy.ndarray

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size=0.3, random_state=42)


X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Validation data shape:", X_val.shape)
print("Validation labels shape:", y_val.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

Training data shape: (42000, 3072)
Training labels shape: (42000, 1)
Validation data shape: (9000, 3072)
Validation labels shape: (9000, 1)
Testing data shape: (9000, 3072)
Testing labels shape: (9000, 1)


In [ ]:
from sklearn.preprocessing import StandardScaler
import cvxopt
class_0_index=np.where(labels==0)[0]#extract indices for which classes  labels are  zero
class_1_index=np.where(labels==1)[0]#extract indices for which classes  labels are  one
combine_lables=np.concatenate((class_0_index,class_1_index))#combine  these indices together for both classes together
x_features=features[combine_lables]#to get features for corrosponding indices of the  classes
y_labels=labels[combine_lables]# to get labels for corrosponding these indices
y_labels=np.where(y_labels==0,-1,1)#class 0->-1 and class 1->+1
scalar=StandardScaler()
x_features_scaled=scalar.fit_transform(x_features)
# Split the data into training and testing sets (70% train, 30% test)
X1_train, X1_test, y1_train, y1_test = train_test_split(x_features_scaled, y_labels, test_size=0.3, random_state=42)


print("Training data shape:", X1_train.shape)
print("Training labels shape:",y1_train.shape)
print("Testing data shape:", X1_test.shape)
print("Testing labels shape:",y1_test.shape)


Training data shape: (8400, 3072)
Training labels shape: (8400, 1)
Testing data shape: (3600, 3072)
Testing labels shape: (3600, 1)


In [ ]:
sample_indices = np.random.choice(len(y1_train), size=3000, replace=False)
X_train_sampled = X1_train[sample_indices]#choose 3000 random samples from first two classes only
y_train_sampled = y1_train[sample_indices]#choose labels for these random samples
K_sampled = np.dot(X_train_sampled, X_train_sampled.T)#x.x.T(kernel)
y_outer_sampled = np.outer(y_train_sampled, y_train_sampled)#y*y
Q = (y_outer_sampled * K_sampled)#element wise multiplication to determine signs of kernal
c = -np.ones(len(y_train_sampled))#is used for subtraction of  linear term summation of alpha
A = y_train_sampled.reshape(1, 3000).astype(float)#a is vector of  y_train_sampled from one row and the same number of coloumns of y_train_sampled
b = np.array([0.0], dtype=float)# just represent vector has zero value represenent equality condition that is summation of alpha multiplied by y=0
C = 1 # Regularization parameter
G = np.vstack([np.eye(len(y_train_sampled)), -np.eye(len(y_train_sampled))])  # g is just compsed of n rows from positive identity matrix and n rows from negative identity matrix(n is number of alpha values)
h = np.hstack([np.ones(len(y_train_sampled)) * C, np.zeros(len(y_train_sampled))])
G # h is composed with n c values and n zero values to compare the first n rows from g that alpha<=c and compare second n rows from g that -alpha<=0

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [-0., -0., -0., ..., -1., -0., -0.],
       [-0., -0., -0., ..., -0., -1., -0.],
       [-0., -0., -0., ..., -0., -0., -1.]])

In [ ]:
Q_cvx = cvxopt.matrix(Q)
c_cvx = cvxopt.matrix(c)
G_cvx = cvxopt.matrix(G)
h_cvx = cvxopt.matrix(h)
A_cvx = cvxopt.matrix(A)
b_cvx = cvxopt.matrix(b)

In [ ]:
import cvxopt
solution = cvxopt.solvers.qp(Q_cvx, c_cvx, G_cvx, h_cvx, A_cvx, b_cvx)

# Extract the optimal alpha values
alpha = np.array(solution['x'])

# Print the results
print("Optimal alpha values:", alpha)
print(alpha.max() , alpha.min())
print("Number of support vectors:", np.sum((alpha > 1e-5) & (alpha < C)))#sum number of support vectors that are not equal 0 and not equal to c as if alpha =c then there is a value to slack variable

     pcost       dcost       gap    pres   dres
 0: -4.5570e+02 -6.3168e+03  3e+04  3e+00  2e-11
 1: -3.2544e+02 -4.1778e+03  9e+03  6e-01  2e-11
 2: -1.9511e+02 -2.1141e+03  3e+03  2e-01  1e-11
 3: -1.0101e+02 -8.5671e+02  1e+03  6e-02  1e-11
 4: -5.6414e+01 -4.1262e+02  5e+02  2e-02  7e-12
 5: -4.1994e+01 -1.6714e+02  2e+02  5e-03  5e-12
 6: -4.2598e+01 -6.5259e+01  2e+01  7e-06  4e-12
 7: -4.6894e+01 -5.4163e+01  7e+00  2e-06  4e-12
 8: -4.8587e+01 -4.9921e+01  1e+00  1e-07  4e-12
 9: -4.8939e+01 -4.9090e+01  2e-01  1e-08  4e-12
10: -4.8984e+01 -4.8989e+01  5e-03  3e-10  4e-12
11: -4.8985e+01 -4.8985e+01  2e-04  1e-11  4e-12
12: -4.8985e+01 -4.8985e+01  3e-06  1e-13  4e-12
Optimal solution found.
Optimal alpha values: [[5.63889638e-11]
 [4.26696837e-11]
 [1.76598963e-05]
 ...
 [1.16299416e-02]
 [9.86595677e-11]
 [4.66438093e-11]]
0.6452726489067503 1.831485706412834e-11
Number of support vectors: 1069


In [ ]:
#  support vectors
import numpy as np
support_vector_indices = np.where((alpha > 1e-5) & (alpha < C))[0]
support_alphas = alpha[support_vector_indices]
support_vectors = X_train_sampled[support_vector_indices]
support_labels = y_train_sampled[support_vector_indices]

print("Number of support vectors:", len(support_vector_indices))


Number of support vectors: 1069


In [ ]:
support_alphas = support_alphas.ravel()
support_labels = support_labels.ravel()

In [ ]:
#  weight vector (w)
w = np.dot(support_alphas * support_labels, support_vectors)
print("Weight vector (w):", w)


Weight vector (w): [ 0.20204767  0.10428764  0.03510375 ...  0.2628892  -0.16879357
  0.19932129]


In [ ]:
# bias (b)
b_values = support_labels - np.dot(support_vectors, w)
b = np.mean(b_values)
print("Bias (b):", b)


Bias (b): -0.2970970321337158


In [ ]:
# Define decision and prediction functions
def decision_function(X):
    return np.dot(X, w) + b

def predict(X):
    return np.sign(decision_function(X))

# Evaluate on test set
test_predictions = predict(X1_test)
accuracy = np.mean(test_predictions == y1_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.5009833333333333


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

# Calculate performance metrics
# Accuracy: Measures the percentage of correct predictions
accuracy = accuracy_score(y1_test, test_predictions)

# F1 Score: Harmonic mean of Precision and Recall, used when there's an imbalance in the classes
f1 = f1_score(y1_test, test_predictions)

# Recall: Measures how well the model detects the positive class
recall = recall_score(y1_test, test_predictions)

# Precision: Measures how accurate the positive predictions are
precision = precision_score(y1_test, test_predictions)

# Print the results
print(f"Accuracy: {accuracy}")  # Prints the accuracy of the model
print(f"F1 Score: {f1}")  # Prints the F1 score, balance between precision and recall
print(f"Recall: {recall}")  # Prints the recall, how well the model detects positive cases
print(f"Precision: {precision}")  # Prints the precision, accuracy of positive predictions

Accuracy: 0.7452777777777778
F1 Score: 0.7252022774947557
Recall: 0.6824591088550479
Precision: 0.7736572890025576
